In [1]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import shapiro, kstest, probplot
from statsmodels.graphics.gofplots import qqplot
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Carga dos Dados 

In [ ]:
# Abrir o dataset e checar a estrutura do mesmo
df_pontuacao = pd.read_csv('./datasets/pontuacao_teste.csv')
df_pontuacao.info()